# Install and import libraries

In [ ]:
!pip install whit-phys-util

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import matplotlib.pyplot as plt
import whit_phys_util as buc

In [ ]:
repo = buc.local_repository("https://github.com/WhitworthPhysics/blackbody-radiation-wiens-law-and-inverse-square-law-maloch18.git")

# Temperature dependence of the blackbody spectrum

## Equipment and materials
| Description | Manufacturer | Model | ID or SN |
| ----------- | ------------ | ----- | -------- |
| radiation sensor | PASCO | TD-8553 | 001369 |
| Stefan-Boltzmann lamp | PASCO | TD-8555 | NAS 1464-062-06 |
| Digital multimeters (2) | Fluke | 117 | 49640101SV, 49640102SV |
| power supply (max. 13 V DC, min. 2 A) | Cenco | 79550 | P-176 |
| spectrometer | Ocean Optics | USB4000 | USB4F07107 |

In [ ]:
R_room = 2.5 # ohms
T_room = 273.15 + 22.0 # K
a = 4.5e-3
R_ref = R_room + a*R_room*(300-T_room)

In [ ]:
df_lamp_calib = pd.read_csv('lamp_calibration.csv')
convert_Rratio_to_T = interp1d(df_lamp_calib['R Ratio'], df_lamp_calib['Temperature (K)'], kind='cubic')

In [ ]:
df_wien = pd.read_table('Wiens_law_data.tsv')
df_wien['log (RS Voltage)'] = np.log(df_wien['Radiation Sensor (mV)'])
df_wien['R (ohms)'] = df_wien['PS Voltage (V)']/df_wien['Current (A)']
df_wien['R ratio'] = df_wien['R (ohms)']/R_ref
df_wien['Temperature (K)'] = convert_Rratio_to_T(df_stefan['R ratio'])
df_wien

In [ ]:
fitTable_wien, fitFunc_wien = buc.prettyPolyFit(1/df_wien['Temperature (K)'], df_wien['Peak wavelength (m)'])
fitTable_wien

In [ ]:
fig_wien, ax_wien = plt.subplots()
ax_wien.plot(1/df_wien['Temperature (K)'], df_wien['Peak wavelength (m)'], '.')
ax_wien.plot(1/df_wien['Temperature (K)'], fitFunc_wien(df_wien['Temperature (K)']))
ax_wien.set_xlabel('$1 \ / \ T \ (\mathrm{K}^{-1})$')
ax_wien.set_ylabel('$\lambda_{\mathrm{peak}}$ (m)');

In [ ]:
b = 2.898e-3
slope_wien = fitTable_wien.iat[0, 0]
slope_unc_wien = fitTable_wien.iat[1, 0]
error_wien = 100 * abs((slope_wien - b) / b)
print("The slope of the wavelength at the emission peak vs. the inverse of the temperature\nwas determined to be ({:.2e} +/- {:.2e}) m*K with an error of {:.1f}%." .format(slope_wien, slope_unc_wien, error_wien))

# Distance dependence of intensity

## Equipment and materials
| Description | Manufacturer | Model | ID or SN |
| ----------- | ------------ | ----- | -------- |
| radiation sensor | PASCO | TD-8553 | 001369 |
| Stefan-Boltzmann lamp | PASCO | TD-8555 | NAS 1464-062-06 |
| Digital multimeters (2) | Fluke | 117 | 49640101SV, 49640102SV |
| power supply (max. 13 V DC, min. 2 A) | Cenco | 79550 | P-176 |
| photodiode detector | Thorlabs | PDA36A | N/A |
| long-pass filter | Thorlabs | FGL610 | N/A |
| meter stick | N/A | N/A | N/A |

In [ ]:
df_dist = pd.read_table('inverse_square_data.tsv')

In [ ]:
def I(x, x0, n, a):
  return a / (x - x0)**n

In [ ]:
df_dist = pd.read_table('inverse_square_data.tsv')
[param, cov] = curve_fit(I, df_dist['x (m)'], df_dist['I (W/m^2)'])
x0 = param[0]
n = param[1]
a = param[2]
print("The values of n and x0 were determined to be {:.1f} and {:.2e} m, respectively." . format(n, x0))

In [ ]:
fig_dist, ax_dist = plt.subplots()
ax_dist.plot(df_dist['x (m)'], df_dist['I (W/m^2)'], '.')
ax_dist.plot(df_dist['x (m)'], I(df_dist['x (m)'], x0, n, a))
ax_dist.set_xlabel('$x$ (m)')
ax_dist.set_ylabel('$I (\mathrm{W} / \ / \mathrm{m}^2$');